In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.set_printoptions(precision=4, linewidth=200)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
print(tf.__version__)

1.3.0


In [3]:
from utils.reader import ptb_raw_data
from utils.batcher import ptb_batcher
from utils.conditional_scope import cond_name_scope, cond_variable_scope
from utils.unrolled_rnn import make_rnn_variables
from utils.unrolled_rnn import make_rnn_outputs
from utils.unrolled_rnn import make_summary_nodes
from utils.unrolled_rnn import make_placeholders
from utils.unrolled_rnn import make_train_op

In [4]:
X_train, X_val, X_test, vocab_size = ptb_raw_data('bigdata/simple-examples/data')

In [5]:
VOCAB_SIZE=128
EMBEDDING_SIZE=8
HIDDEN_SIZE=12
BATCH_SIZE=4
NUM_STEPS=16

tf.reset_default_graph()
placeholders = make_placeholders(
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS
)
rnn_vars = make_rnn_variables(
    vocab_size=VOCAB_SIZE,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
)
rnn_outputs = make_rnn_outputs(
    input_sequence=placeholders['inputs'],
    vocab_size=VOCAB_SIZE,
    hidden_size=HIDDEN_SIZE,
    batch_size=BATCH_SIZE,
    num_steps=NUM_STEPS,
    rnn_variables=rnn_vars
)
summary_nodes = make_summary_nodes(
    targets=placeholders['targets'],
    logits=rnn_outputs['logits'],
)
train_op = make_train_op(
    summary_nodes['loss'],
    placeholders['learning_rate'],
)

In [8]:
inputs = np.random.randint(VOCAB_SIZE, size=(BATCH_SIZE, NUM_STEPS))
targets = np.random.randint(VOCAB_SIZE, size=(BATCH_SIZE, NUM_STEPS))
training_outputs = {**summary_nodes, 'train_op': train_op}
with tf.Session() as sess:
    run_id = time.time()
    writer = tf.summary.FileWriter('logs/{0}'.format(run_id), sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        outputs = sess.run(
            training_outputs,
            feed_dict={
                placeholders['inputs']: inputs,
                placeholders['targets']: targets,
            }
        )
        if i % 100 == (100 - 1):
            print(outputs['loss'], outputs['num_correct_predictions'])
    params = sess.run(
        rnn_vars
    )
    print(params)
    writer.close()

12.711 6
{'embedding_matrix': array([[-0.0258, -0.0133,  0.0026, ...,  0.0323, -0.0352, -0.0554],
       [-0.0411, -0.0204, -0.0396, ..., -0.0342,  0.0051,  0.0269],
       [-0.0249,  0.0251, -0.0011, ...,  0.046 ,  0.0143, -0.0269],
       ..., 
       [-0.0353,  0.0226,  0.0026, ..., -0.046 ,  0.0189, -0.0049],
       [-0.0177,  0.0475,  0.0298, ..., -0.0447, -0.0084,  0.0401],
       [-0.0081, -0.0197,  0.0204, ..., -0.0436, -0.033 ,  0.0389]], dtype=float16), 'gru_params': {'U_z': array([[-0.0294,  0.0319,  0.1666,  0.1829,  0.1307,  0.1089, -0.0081,  0.1688, -0.7593,  0.1404,  0.0559, -0.4912],
       [-0.0603, -0.3569,  0.0842,  0.1373,  0.1309,  0.0645, -0.3101,  0.0704, -0.2546, -0.1038, -0.3494, -0.194 ],
       [ 0.1541,  0.0787,  0.2839,  0.3137,  0.2639,  0.252 ,  0.0528,  0.3057,  0.0885,  0.2905,  0.1865,  0.105 ],
       [-0.1593, -0.1038, -0.324 , -0.3186, -0.2778, -0.342 , -0.0689, -0.2644, -0.1072, -0.2573, -0.1544, -0.0514],
       [ 0.1304,  0.0591,  0.2776,  0.3274